# Biodiversity Intactness Index change in Phoenix, AZ

In [1]:
# Import necessary libraries along with defined abbreviations
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import geopandas as gpd 
import xarray as xarray

In [4]:
# Set up display options when looking at data frames
pd.set_option("display.max.columns", None)


## Import shapefiles
# Establish file path to read in shapefile
az_fp = os.path.join("data", "tl_2025_04_tract", 
                        "tl_2025_04_tract.shp")
az = gpd.read_file(az_fp)
